In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import preprocessing
import umap
from matplotlib.backends.backend_pdf import PdfPages
import scipy.stats
warnings.filterwarnings("ignore")
plt.rcParams["font.size"] = 10
from tsfresh import extract_features
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests


In [4]:

########## Load each data frame into dictionary - Permian ##########
file_names = ! ls permian_despiked_120523
dict_dfs = {}
for file_name in file_names:
    df=pd.read_csv(f'permian_despiked_120523/{file_name}')
    # remove_max=df[df['DEPTH(ft)']<=30].index.max()
    # array =np.empty((remove_max+1))
    # array[:] = np.nan
    # new_gamma = list(array) + list(df['Gamma(gapi)']) 
    # new_gamma= new_gamma[:-remove_max-1]
    # df['Gamma(gapi)_unlagged30'] = new_gamma
    dict_dfs[file_name]=df
    
# these are the cols of interest
consistent_col_names =['DEPTH(ft)',
 'ROP(ft / min)',
 'WOB(klbs)',
 'TPO(gal / min)',
 'RPM(rpm)',
 'SPP(psi)',
 'INC(deg)',
 'Gamma(gapi)',
 'DIFP(psi)']

df=dict_dfs[file_names[0]]
df=df[consistent_col_names].dropna() 

In [5]:


# get horz and vertical drilling separately
df_horz=df[df['INC(deg)']>80]
df_vert=df[df['INC(deg)']<=80]
# create ids to represent different windows -50ft
def add_ids(df):
    ids=[]
    window=60 #equivalent to 30ft
    id=0
    for i in range(1,len(df)+1):
        if (i/window).is_integer():
            id=i/window
            ids.append(id)
        else: 
            ids.append(id)
    df['id']=ids
    small_window=pd.Series(ids).value_counts()[pd.Series(ids).value_counts()<99].index[0]
    df= df[df['id']!=small_window] #remove small window
    df['id']=df['id'].astype(int)
    return df
df_horz=add_ids(df_horz)
df_vert=add_ids(df_vert)

# TODO consider nan handling
# df_features = extract_features(df_horz.dropna(), column_id="id", column_sort="DEPTH(ft)")


## extract features at a window

In [4]:
fc_settings = {'abs_energy': None}
# fc_settings = {'variance_larger_than_standard_deviation': None,
#  'has_duplicate_max': None,
#  'has_duplicate_min': None,
#  'has_duplicate': None,
#  'sum_values': None,
#  'abs_energy': None,
#  'mean_abs_change': None,
#  'mean_change': None,
#  'mean_second_derivative_central': None,
#  'median': None,
#  'mean': None,
#  'length': None,
#  'standard_deviation': None,
#  'variation_coefficient': None,
#  'variance': None,
#  'skewness': None,
#  'kurtosis': None,
#  'root_mean_square': None,
#  'absolute_sum_of_changes': None,
#  'longest_strike_below_mean': None,
#  'longest_strike_above_mean': None,
#  'count_above_mean': None,
#  'count_below_mean': None,
#  'last_location_of_maximum': None,
#  'first_location_of_maximum': None,
#  'last_location_of_minimum': None,
#  'first_location_of_minimum': None,
#  'percentage_of_reoccurring_values_to_all_values': None,
#  'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
#  'sum_of_reoccurring_values': None,
#  'sum_of_reoccurring_data_points': None,
#  'ratio_value_number_to_time_series_length': None,
#  'maximum': None,
#  'minimum': None,
#  'benford_correlation': None,
#  'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
#  'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
#  'cid_ce': [{'normalize': True}, {'normalize': False}],
#  'symmetry_looking': [{'r': 0.0},
#    {'r': 0.1},
#   {'r': 0.2},
#   {'r': 0.30000000000000004},
#   {'r': 0.4},
#   {'r': 0.5}],
#  'large_standard_deviation': [{'r': 0.5},
#   {'r': 0.75},
#   {'r': 0.9500000000000001}],
#  'quantile': [{'q': 0.1},
#   {'q': 0.2},
#   {'q': 0.3},
#   {'q': 0.4},
#   {'q': 0.6},
#   {'q': 0.7},
#   {'q': 0.8},
#   {'q': 0.9}],
#  'autocorrelation': [{'lag': 0},
#   {'lag': 1},
#   {'lag': 2},
#   {'lag': 3},
#   {'lag': 4},
#   {'lag': 5},
#   {'lag': 6},
#   {'lag': 7},
#   {'lag': 8},
#   {'lag': 9}],
#  'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
#   {'f_agg': 'median', 'maxlag': 40},
#   {'f_agg': 'var', 'maxlag': 40}],
#  'partial_autocorrelation': [{'lag': 0},
#   {'lag': 1},
#   {'lag': 2},
#   {'lag': 3},
#   {'lag': 4},
#   {'lag': 5},
#   {'lag': 6},
#   {'lag': 7},
#   {'lag': 8},
#   {'lag': 9}],
#  'number_cwt_peaks': [{'n': 1}, {'n': 5}],
#  'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}, {'n': 10}, {'n': 50}],
#  'binned_entropy': [{'max_bins': 10}],
#  'index_mass_quantile': [{'q': 0.1},
#   {'q': 0.2},
#   {'q': 0.3},
#   {'q': 0.4},
#   {'q': 0.6},
#   {'q': 0.7},
#   {'q': 0.8},
#   {'q': 0.9}],
#  'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
#  'ar_coefficient': [{'coeff': 0, 'k': 10},
#   {'coeff': 1, 'k': 10},
#   {'coeff': 2, 'k': 10},
#   {'coeff': 3, 'k': 10},
#   {'coeff': 4, 'k': 10},
#   {'coeff': 5, 'k': 10},
#   {'coeff': 6, 'k': 10},
#   {'coeff': 7, 'k': 10},
#   {'coeff': 8, 'k': 10},
#   {'coeff': 9, 'k': 10},
#   {'coeff': 10, 'k': 10}],
#  'value_count': [{'value': 0}, {'value': 1}, {'value': -1}],
#  'range_count': [{'min': -1, 'max': 1}],
#  'linear_trend': [{'attr': 'pvalue'},
#   {'attr': 'rvalue'},
#   {'attr': 'intercept'},
#   {'attr': 'slope'},
#   {'attr': 'stderr'}],
#  'augmented_dickey_fuller': [{'attr': 'teststat'},
#   {'attr': 'pvalue'},
#   {'attr': 'usedlag'}],
#  'number_crossing_m': [{'m': 0}, {'m': -1}, {'m': 1}],
#  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
#   {'num_segments': 10, 'segment_focus': 1},
#   {'num_segments': 10, 'segment_focus': 2},
#   {'num_segments': 10, 'segment_focus': 3},
#   {'num_segments': 10, 'segment_focus': 4},
#   {'num_segments': 10, 'segment_focus': 5},
#   {'num_segments': 10, 'segment_focus': 6},
#   {'num_segments': 10, 'segment_focus': 7},
#   {'num_segments': 10, 'segment_focus': 8},
#   {'num_segments': 10, 'segment_focus': 9}],
#  'ratio_beyond_r_sigma': [{'r': 0.5},
#   {'r': 1},
#   {'r': 1.5},
#   {'r': 2},
#   {'r': 2.5},
#   {'r': 3},
#   {'r': 5},
#   {'r': 6},
#   {'r': 7},
#   {'r': 10}],
#  'linear_trend_timewise': [{'attr': 'pvalue'},
#   {'attr': 'rvalue'},
#   {'attr': 'intercept'},
#   {'attr': 'slope'},
#   {'attr': 'stderr'}],
#  'count_above': [{'t': 0}],
#  'count_below': [{'t': 0}],
#  'permutation_entropy': [{'tau': 1, 'dimension': 3},
#   {'tau': 1, 'dimension': 4},
#   {'tau': 1, 'dimension': 5},
#   {'tau': 1, 'dimension': 6},
#   {'tau': 1, 'dimension': 7}],
#  'query_similarity_count': [{'query': None, 'threshold': 0.0}]}
 

In [6]:
df_features = extract_features(df_horz, column_id="id", column_sort="DEPTH(ft)")
df_features.columns


Feature Extraction: 100%|██████████| 10/10 [00:36<00:00,  3.68s/it]


Index(['ROP(ft / min)__variance_larger_than_standard_deviation',
       'ROP(ft / min)__has_duplicate_max', 'ROP(ft / min)__has_duplicate_min',
       'ROP(ft / min)__has_duplicate', 'ROP(ft / min)__sum_values',
       'ROP(ft / min)__abs_energy', 'ROP(ft / min)__mean_abs_change',
       'ROP(ft / min)__mean_change',
       'ROP(ft / min)__mean_second_derivative_central',
       'ROP(ft / min)__median',
       ...
       'DIFP(psi)__fourier_entropy__bins_5',
       'DIFP(psi)__fourier_entropy__bins_10',
       'DIFP(psi)__fourier_entropy__bins_100',
       'DIFP(psi)__permutation_entropy__dimension_3__tau_1',
       'DIFP(psi)__permutation_entropy__dimension_4__tau_1',
       'DIFP(psi)__permutation_entropy__dimension_5__tau_1',
       'DIFP(psi)__permutation_entropy__dimension_6__tau_1',
       'DIFP(psi)__permutation_entropy__dimension_7__tau_1',
       'DIFP(psi)__query_similarity_count__query_None__threshold_0.0',
       'DIFP(psi)__mean_n_absolute_max__number_of_maxima_7'],
      

In [96]:
df_features


,RPM(rpm)__abs_energy,SPP(psi)__abs_energy,INC(deg)__abs_energy,ROP(ft / min)__abs_energy,WOB(klbs)__abs_energy,TPO(gal / min)__abs_energy,Gamma(gapi)__abs_energy,DIFP(psi)__abs_energy,BIT(in)__abs_energy
0,60164.937407,5.087632e+08,666978.823509,36.946027,8847.432593,6.007195e+06,1.290071e+06,5.044432e+06,3719.1331
1,64066.897500,5.087873e+08,731760.778589,48.051957,21474.247500,6.270076e+06,1.282699e+06,4.470001e+06,3757.3225
2,100030.818750,4.953173e+08,785279.400881,137.806653,9222.958750,6.547273e+06,1.013324e+06,3.729697e+06,3756.9550


In [104]:
# # pd.Series(df_features.columns[df_features.columns.str.contains('ROP')].str.replace('ROP(ft / min)__','')).to_csv('features.csv')
cols=list(df_features.columns[df_features.columns.str.contains('ROP')])
# cols

## Assess granger causality on raw features to see if they predict gamma - 

In [61]:
df_horz

,DEPTH(ft),ROP(ft / min),WOB(klbs),TPO(gal / min),RPM(rpm),SPP(psi),INC(deg),Gamma(gapi),DIFP(psi),BIT(in),Gamma(gapi)_unlagged30,id
21146,10573.0,0.183333,13.700000,246.000000,0.100000,2188.97,80.050000,84.3804,238.070000,6.13,102.4234,0
21147,10573.5,0.238333,14.100000,246.000000,0.100000,2227.59,80.101562,75.5345,276.590000,6.13,106.5349,0
21148,10574.0,0.266667,13.100000,246.000000,0.100000,2243.29,80.153125,72.2824,292.220000,6.13,109.7973,0
21149,10574.5,0.288333,12.500000,246.000000,0.100000,2225.67,80.204688,71.9504,274.720000,6.13,113.7333,0
21150,10575.0,0.316111,12.055556,245.888889,3.977778,2232.89,80.256250,72.2764,270.604444,6.13,117.1055,0
...,...,...,...,...,...,...,...,...,...,...,...,...
29790,14895.0,0.340000,24.700000,221.000000,4.000000,3030.07,89.865263,75.6183,265.640000,6.13,75.6183,144
29791,14895.5,0.295000,11.200000,221.000000,58.100000,3165.30,89.843684,75.6183,400.680000,6.13,75.6183,144
29792,14896.0,0.408333,11.500000,221.000000,59.100000,3110.84,89.822105,75.6183,346.250000,6.13,75.6183,144
29793,14896.5,0.886667,9.900000,221.000000,58.900000,3197.25,89.800526,75.6183,432.630000,6.13,75.6183,144


In [69]:
features = df_horz.columns[1:-1]
col=features[1]
result = adfuller(df[col])
print(col)
print(f'Test Statistics: {result[0]}')
print(f'p-value: {result[1]}')
print(f'critical_values: {result[4]}')
if result[1] > 0.05:
    print("Series is not stationary")
else:
    print("Series is stationary")

WOB(klbs)
Test Statistics: -11.082603284570562
p-value: 4.2740602017679e-20
critical_values: {'1%': -3.4310877841541583, '5%': -2.861866052116409, '10%': -2.5669435525409803}
Series is stationary


In [70]:
gc = grangercausalitytests(df_horz[['Gamma(gapi)', col]], maxlag=50,verbose=False)


In [71]:
lag_pvalues={}
for k,v in gc.items():
    if k>=30:
        lag_pvalues[k]=v[0]['params_ftest'][1]
lag_pvalues

{30: 0.8889434930977717,
 31: 0.7957190610037227,
 32: 0.7965257645013659,
 33: 0.7909090955013794,
 34: 0.753822083873425,
 35: 0.7856110199998049,
 36: 0.8173394142692672,
 37: 0.8471900045760425,
 38: 0.8481231948561001,
 39: 0.8556160803828587,
 40: 0.7337997136876182,
 41: 0.7446538593197431,
 42: 0.7617976894032865,
 43: 0.7118565863634836,
 44: 0.6458006983542526,
 45: 0.5643707465694061,
 46: 0.5653777352254485,
 47: 0.586556759937316,
 48: 0.5509686283103107,
 49: 0.39443141854668745,
 50: 0.42987999000695043}